# Continue Pretraining on T5

In [1]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('../../Models/my_tokenizer')

In [2]:
%env WANDB_PROJECT=english-v2

env: WANDB_PROJECT=english-v2


In [3]:
from transformers import TrainingArguments, Trainer, T5ForConditionalGeneration, Adafactor
from accelerate import Accelerator

model = T5ForConditionalGeneration.from_pretrained("chkpts/chkpt-1080000", device_map='cuda:0')
model.resize_token_embeddings(len(tokenizer))

accelerator = Accelerator()

optimizer = Adafactor(
    model.parameters(), 
    scale_parameter=False, 
    relative_step=False, 
    warmup_init=False, 
    lr=3e-4
)

model, optimizer = accelerator.prepare(model, optimizer)

training_args = TrainingArguments(
    output_dir="buddhist-large-pretrain",
    auto_find_batch_size=True,
    learning_rate=3e-4,
    num_train_epochs=1,
    save_strategy='no'
    )

In [5]:
from datasets import load_dataset
import gc

for i in range(4, 5):
    
    start = 1080000
    end = 1722833

    print(f'Training on {start} to {end}')

    tokenized_dataset = load_dataset('billingsmoore/temp2',split='train').select(range(start, end))
    gc.collect()

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
        optimizers=(optimizer, None)
    )

    trainer.train()

    model.save_pretrained(f'chkpts/chkpt-{end}')

    del tokenized_dataset
    gc.collect()

Using the latest cached version of the dataset since billingsmoore/temp2 couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/j/.cache/huggingface/datasets/billingsmoore___temp2/default/0.0.0/bd0498d038398366dcc696b3f79337ca96cdbb27 (last modified on Sun May 25 21:01:21 2025).


Training on 1080000 to 1722833


Loading dataset shards:   0%|          | 0/17 [00:00<?, ?it/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: billingsmoore. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
500,0.124200
1000,0.117700
1500,0.120000
2000,0.127000
2500,0.120300
3000,0.124100
3500,0.129800
4000,0.127600
4500,0.119200
5000,0.127300


In [6]:
model.save_pretrained('pretrained-large-model')